In [4]:
import sqlite3
import pandas as pd

In [5]:
# Connect to the SQLite databases
conn1 = sqlite3.connect('songs1.sqlite')
conn2 = sqlite3.connect('songs2.sqlite')
conn3 = sqlite3.connect('songs3.sqlite')
conn4 = sqlite3.connect('songs4.sqlite')
conn5 = sqlite3.connect('songs5.sqlite')

# Load tables into pandas DataFrames
table_v1 = pd.read_sql_query("SELECT * FROM songs", conn1)
table_v2 = pd.read_sql_query("SELECT * FROM songs", conn2)
table_v3 = pd.read_sql_query("SELECT * FROM songs", conn3)
table_v4 = pd.read_sql_query("SELECT * FROM songs", conn4)
table_v5 = pd.read_sql_query("SELECT * FROM songs", conn5)

Column comparison

In [6]:
# find deleted columns
missing_cols = [column for column in table_v1.columns if column not in table_v2.columns]

# find added columns
added_cols = [column for column in table_v2.columns if column not in table_v1.columns]

# find common columns 
common_cols = [column for column in table_v1.columns if column in table_v2.columns]

print('common: ', common_cols)
print('added: ', added_cols)
print('removed: ', missing_cols)

common:  ['artist_names', 'track_name', 'source', 'tempo', 'weeks_on_chart', 'streams']
added:  ['mode', 'loudness']
removed:  ['id', 'duration_ms']


In [7]:
# combine versions to also display deleted rows 
comparison = pd.merge(table_v1, table_v2, on = common_cols, how= 'outer')

In [8]:
def apply_color(col, color):
    return [f'background-color: {color}' for c in col]

styled_df = comparison.head(20).style.apply(apply_color, color='red', subset=pd.IndexSlice[:, missing_cols])\
                     .apply(apply_color, color='green', subset=pd.IndexSlice[:, added_cols])

Highlight added cloumns green and deleted ones red 

In [9]:
styled_df

,id,artist_names,track_name,source,tempo,duration_ms,weeks_on_chart,streams,mode,loudness
0,000xQL6tZNLJzIrtIgxqSl,"""ZAYN","PARTYNEXTDOOR""",Still Got Time (feat. PARTYNEXTDOOR),-6.029000,120.963000,188491.000000,17.000000,G,0.524000
1,003eoIwxETJujVWmNFMoZy,Alessia Cara,Growing Pains,Def Jam Recordings,191.153000,193680.000000,2.000000,9944865.000000,Minor,-6.276000
2,003vvx7Niy0yvhvHt4a68B,The Killers,Mr. Brightside,Island Records,148.033000,222973.000000,125.000000,512388123.000000,Major,-5.230000
3,00B7TZ0Xawar6NZ00JFomN,"""Cardi B","Chance the Rapper""",Best Life (feat. Chance The Rapper),-7.438000,167.911000,284856.000000,2.000000,A,0.665000
4,00Blm7zeNqgYLPtW6zg8cj,"""Post Malone","The Weeknd""",One Right Now (with The Weeknd),-4.806000,97.014000,193507.000000,30.000000,C#/Db,0.688000
5,00EPIEnX1JFjff8sC6bccd,"""Thalia","NATTI NATASHA""",No Me Acuerdo,-4.247000,94.033000,217653.000000,16.000000,G,0.772000
6,00ETaeHUQ6lops3oWU1Wrt,"""Kygo","Donna Summer""",Hot Stuff,-5.749000,119.961000,199008.000000,1.000000,F,0.429000
7,00ZKeP47bZtswtANkvxz2j,"""Tropa do Bruxo",DJ Ws da Igrejinha,SMU,0.006420,0.102000,0.522000,-4.731000,"Mc Menor Thalis""",0.889000
8,00gpGR84M27moP7AFuqHIx,YBN Nahmir,Bounce Out With That,2018,94.949000,91011.000000,6.000000,4913180.000000,Major,-8.278000
9,00imgaPlYRrMGn9o83hfmk,Brent Faiyaz,LOOSE CHANGE,"""Lost Kids LLC.",-8.931000,84.975000,226011.000000,1.000000,C#/Db,0.440000


Pandas compare function (works only for identically-labeled (both index and columns) DataFrame objects)

In [10]:
table_v4.compare(table_v3)

artist_names              
           self         other
13  Helena Kohl  XXXTENTACION

In [11]:
table_v4.compare(table_v3, keep_shape = True)

artist_names       track_name       source       mode       loudness  \
            self other       self other   self other self other     self   
0            NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
1            NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
2            NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
3            NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
4            NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
..           ...   ...        ...   ...    ...   ...  ...   ...      ...   
988          NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
989          NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
990          NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
991          NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   
992          NaN   NaN        NaN   NaN    NaN   NaN  NaN   NaN      NaN   

          tempo       weeks_on_chart       streams        
    other  self other           self other    self other  
0     NaN   NaN   NaN            NaN   NaN     NaN   NaN  
1     NaN   NaN   NaN            NaN   NaN     NaN   NaN  
2     NaN   NaN   NaN            NaN   NaN     NaN   NaN  
3     NaN   NaN   NaN            NaN   NaN     NaN   NaN  
4     NaN   NaN   NaN            NaN   NaN     NaN   NaN  
..    ...   ...   ...            ...   ...     ...   ...  
988   NaN   NaN   NaN            NaN   NaN     NaN   NaN  
989   NaN   NaN   NaN            NaN   NaN     NaN   NaN  
990   NaN   NaN   NaN            NaN   NaN     NaN   NaN  
991   NaN   NaN   NaN            NaN   NaN     NaN   NaN  
992   NaN   NaN   NaN            NaN   NaN     NaN   NaN  

[993 rows x 16 columns]

In [12]:
table_v1.equals(table_v1)

True

In [13]:
table_v1.equals(table_v3)

False

Datacompy

In [14]:
import datacompy

In [15]:
compare = datacompy.Compare(
    table_v1,
    table_v4,
    join_columns= ('artist_names','track_name', 'source'),  #You can also specify a list of columns
    df1_name='Original', 
    df2_name='New' 
    )
compare.matches(ignore_extra_columns=False)

print(compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  Original        8   998
1       New        8   993

Column Summary
--------------

Number of columns in common: 6
Number of columns in Original but not in New: 2
Number of columns in New but not in Original: 2

Row Summary
-----------

Matched on: artist_names, track_name, source
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 992
Number of rows in Original but not in New: 6
Number of rows in New but not in Original: 1

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 992

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 6
Total number of values which compare unequal: 0

Sample Rows Only in Original (First 10 Columns)
-----------------------------------------------

            

In [16]:
compare2 = datacompy.Compare(
    table_v2,
    table_v4,
    join_columns= ('artist_names','track_name', 'source'),  #You can also specify a list of columns
    df1_name='Original', 
    df2_name='New' 
    )
compare2.matches(ignore_extra_columns=False)

print(compare2.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  Original        8   998
1       New        8   993

Column Summary
--------------

Number of columns in common: 8
Number of columns in Original but not in New: 0
Number of columns in New but not in Original: 0

Row Summary
-----------

Matched on: artist_names, track_name, source
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 992
Number of rows in Original but not in New: 6
Number of rows in New but not in Original: 1

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 992

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 8
Total number of values which compare unequal: 0

Sample Rows Only in Original (First 10 Columns)
-----------------------------------------------

         art